In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Acre - Produção de Cimento (t)', axis=1)
data

,Ano,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Consumo de Cimento (t)
0,2008,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.691749,0.773119,0.647836,0.657765,...,2.868108,11.760983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.145000
1,2009,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.695139,0.777398,0.653540,0.658635,...,2.305885,9.472585,27.667957,984.971935,1299.760655,25.376339,21.219990,79.673793,1.436413,121.146000
2,2010,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.698459,0.781539,0.659233,0.659404,...,1.958246,9.332485,30.587558,985.037587,1305.345781,25.288855,17.738219,70.049719,1.246421,174.705000
3,2011,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.701724,0.785565,0.664930,0.660077,...,2.079088,10.985689,34.156125,985.282207,1317.710216,25.242893,16.188303,66.211701,1.234468,178.322000
4,2012,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.709974,0.795847,0.678860,0.661919,...,2.183282,8.159014,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205,173.494000
5,2013,11.122668,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.710721,0.796251,0.681349,0.661246,...,2.403445,7.913561,25.635353,985.492097,1244.679422,26.085119,20.000566,73.871198,1.665816,193.672000
6,2014,11.666939,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.711115,0.796188,0.683294,0.660516,...,2.748397,10.350442,23.987188,985.694959,1230.934473,26.457083,21.165984,76.068571,1.660711,171.709286
7,2015,12.139670,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.720255,0.808026,0.697137,0.663153,...,2.501529,12.454398,28.288853,986.021421,1278.252041,26.043363,21.489384,78.704494,1.603219,151.486629
8,2016,12.458072,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.720460,0.807937,0.698785,0.662167,...,2.073464,13.111908,31.329110,985.952489,1333.799567,25.936106,21.376188,78.087996,1.551043,134.223086
9,2017,12.245790,8.632052e+06,389052.079088,9.235469,6.808242e+06,0.720230,0.807295,0.698311,0.662364,...,2.177386,9.479812,32.173955,987.415055,1385.250388,26.527832,21.280819,74.803746,1.421881,150.491000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.691749,0.773119,0.647836,0.657765,5.483335e+08,...,-7723.574973,2.868108,11.760983,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.695139,0.777398,0.653540,0.658635,5.302764e+08,...,-8961.781709,2.305885,9.472585,27.667957,984.971935,1299.760655,25.376339,21.219990,79.673793,1.436413
2,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.698459,0.781539,0.659233,0.659404,5.046425e+08,...,-9895.682385,1.958246,9.332485,30.587558,985.037587,1305.345781,25.288855,17.738219,70.049719,1.246421
3,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.701724,0.785565,0.664930,0.660077,4.854622e+08,...,-9989.170714,2.079088,10.985689,34.156125,985.282207,1317.710216,25.242893,16.188303,66.211701,1.234468
4,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.709974,0.795847,0.678860,0.661919,4.867661e+08,...,-9429.326328,2.183282,8.159014,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205
5,11.122668,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.710721,0.796251,0.681349,0.661246,5.225848e+08,...,-7157.441850,2.403445,7.913561,25.635353,985.492097,1244.679422,26.085119,20.000566,73.871198,1.665816
6,11.666939,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.711115,0.796188,0.683294,0.660516,5.676413e+08,...,-3453.461327,2.748397,10.350442,23.987188,985.694959,1230.934473,26.457083,21.165984,76.068571,1.660711
7,12.139670,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.720255,0.808026,0.697137,0.663153,5.179392e+08,...,-1442.919387,2.501529,12.454398,28.288853,986.021421,1278.252041,26.043363,21.489384,78.704494,1.603219
8,12.458072,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.720460,0.807937,0.698785,0.662167,4.524144e+08,...,-3128.377228,2.073464,13.111908,31.329110,985.952489,1333.799567,25.936106,21.376188,78.087996,1.551043
9,12.245790,8.632052e+06,389052.079088,9.235469,6.808242e+06,0.720230,0.807295,0.698311,0.662364,4.213689e+08,...,-5386.337108,2.177386,9.479812,32.173955,987.415055,1385.250388,26.527832,21.280819,74.803746,1.421881


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     121.146000
1     174.705000
2     178.322000
3     173.494000
4     193.672000
5     171.709286
6     151.486629
7     134.223086
8     150.491000
9     148.696000
10    153.806000
11    178.033000
12    189.357000
13    202.321000
14           NaN
Name: Acre - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input = train_input.fillna(0.0)
train_input

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.334957,-1.852442,1.525597,1.304983,-1.799100,-1.762544,-1.803010,-1.732102,-1.948153,1.204752,...,-0.569731,2.026352,0.984445,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1.317912,-1.694326,1.677576,1.094681,-1.854414,-1.436150,-1.450052,-1.428386,-1.426073,0.820657,...,-0.838980,0.085558,-0.051710,-0.787524,-0.785565,-0.154931,-0.941087,0.514620,1.432621,0.152515
2,-1.335181,-1.104775,1.084964,0.895963,-0.871808,-1.116609,-1.108493,-1.125232,-0.964539,0.275392,...,-1.042057,-1.114490,-0.115145,0.000324,-0.755268,-0.025100,-1.049453,-1.293368,-1.046132,-0.740078
3,-1.343483,-0.560051,0.614372,0.794584,-0.061186,-0.802204,-0.776443,-0.821884,-0.560517,-0.132596,...,-1.062386,-0.697344,0.633403,0.963296,-0.642379,0.262321,-1.106386,-2.098196,-2.034643,-0.796236
4,-0.216182,-0.050154,0.319763,0.747334,0.727969,-0.008074,0.071682,-0.080164,0.544982,-0.104860,...,-0.940648,-0.337667,-0.646476,0.294792,-0.542399,-0.205077,-0.844884,-1.199374,-0.965077,0.320666
5,0.268317,0.382937,0.024542,0.585657,1.261692,0.063916,0.104998,0.052331,0.140808,0.657046,...,-0.446625,0.422337,-0.757614,-1.336019,-0.545518,-1.435339,-0.063118,-0.118593,-0.061881,1.230260
6,0.602908,0.693077,-0.216274,0.203003,1.333947,0.101792,0.099846,0.155932,-0.297048,1.615453,...,0.358807,1.613108,0.345772,-1.780773,-0.451900,-1.754851,0.397635,0.486576,0.504069,1.206277
7,0.893519,0.807496,-0.923301,-0.593627,0.680859,0.981753,1.076307,0.892972,1.285447,0.558230,...,0.796000,0.760923,1.298413,-0.619977,-0.301242,-0.654916,-0.114841,0.654509,1.182971,0.936177
8,1.089257,0.882491,-1.156767,-1.099436,0.293116,1.001492,1.069003,0.980763,0.693593,-0.835563,...,0.429496,-0.716758,1.596124,0.200431,-0.333053,0.636331,-0.247701,0.595729,1.024187,0.691049
9,0.958757,0.917863,-1.156526,-1.258364,0.252392,0.979266,1.016042,0.955522,0.811815,-1.495939,...,-0.061498,-0.358019,-0.048438,0.428411,0.341900,1.832347,0.485273,0.546207,0.178303,0.084240


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     121.146000
1     174.705000
2     178.322000
3     173.494000
4     193.672000
5     171.709286
6     151.486629
7     134.223086
8     150.491000
9     148.696000
10    153.806000
11    178.033000
Name: Acre - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.704977,0.401023,-0.568106,-1.099064,-0.414155,0.731364,0.557172,0.863927,0.163877,1.108405,...,0.964765,-1.482779,-2.260447,1.033247,1.639674,0.502837,2.461657,1.121007,-1.165178,-2.121943


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    189.357
Name: Acre - Consumo de Cimento (t), dtype: float64

In [31]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1024, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [34]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [35]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[381362180, 3666628198, 455916598, 869952658, 1338790126, 2613047908, 2788709675, 3219378897, 3334185777, 1689632446]


Step: 0 ___________________________________________
val_loss: 637.2828369140625
winner_seed: 381362180


Step: 1 ___________________________________________
val_loss: 485.0028991699219
winner_seed: 3666628198


Step: 2 ___________________________________________
val_loss: 66.10963439941406
winner_seed: 455916598


Step: 3 ___________________________________________
val_loss: 0.00023798109032213688
winner_seed: 869952658


Step: 4 ___________________________________________
val_loss: 492.3372802734375


Step: 5 ___________________________________________
val_loss: 307.6278076171875


Step: 6 ___________________________________________
val_loss: 0.03999267518520355


Step: 7 ___________________________________________
val_loss: 38.656341552734375


Step: 8 ___________________________________________
val_loss: 528.37548828125


Step: 9 ___________________________________

In [36]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 474ms/step - loss: 26233.0684 - val_loss: 31422.7285
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 24912.5996 - val_loss: 22436.9141
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 24715.4434 - val_loss: 41687.5195
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 23883.4004 - val_loss: 40131.1445
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 23164.8457 - val_loss: 38578.1914
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 22092.2422 - val_loss: 39942.3008
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 20599.1895 - val_loss: 29122.0293
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 19275.0059 - val_loss: 34887.4766
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 21455.0215 - val_loss: 30175.8242
Epoch 10/10000
1/1 [=======

1/1 [==============================] - 0s 14ms/step - loss: 6135.9277 - val_loss: 23024.7559
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 6105.5493 - val_loss: 21248.6230
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 5591.0840 - val_loss: 19647.3242
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 5243.7090 - val_loss: 18336.7246
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 5106.8486 - val_loss: 17512.6152
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 4753.8979 - val_loss: 17581.2812
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 4708.3276 - val_loss: 18497.4688
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 4520.0527 - val_loss: 14260.9502
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 4304.7163 - val_loss: 13994.2500
Epoch 85/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 2620.3938 - val_loss: 8664.9463
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 2588.7424 - val_loss: 8757.5596
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 2485.3494 - val_loss: 9851.3350
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 2545.2686 - val_loss: 9300.6318
Epoch 155/10000
1/1 [==============================] - 0s 15ms/step - loss: 2503.0315 - val_loss: 8980.2402
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 2469.9324 - val_loss: 8777.1113
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 2439.6191 - val_loss: 8633.0029
Epoch 158/10000
1/1 [==============================] - 0s 16ms/step - loss: 2411.0388 - val_loss: 8517.2881
Epoch 159/10000
1/1 [==============================] - 0s 15ms/step - loss: 2383.1426 - val_loss: 8416.1807
Epoch 160/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 1160.6469 - val_loss: 4288.3853
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 1149.5776 - val_loss: 4248.0425
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 1138.6525 - val_loss: 4208.1270
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 1127.8677 - val_loss: 4168.6631
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 1117.2236 - val_loss: 4129.6147
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 1106.7164 - val_loss: 4091.0171
Epoch 233/10000
1/1 [==============================] - 0s 15ms/step - loss: 1096.3462 - val_loss: 4052.8389
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 1086.1097 - val_loss: 4015.0891
Epoch 235/10000
1/1 [==============================] - 0s 14ms/step - loss: 1076.0317 - val_loss: 3977.6160
Epoch 236/10000
1/1 [=======================

1/1 [==============================] - 0s 13ms/step - loss: 623.2899 - val_loss: 2198.5747
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 619.1777 - val_loss: 2180.9324
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 615.1211 - val_loss: 2163.4778
Epoch 306/10000
1/1 [==============================] - 0s 13ms/step - loss: 611.1152 - val_loss: 2146.2090
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 607.1472 - val_loss: 2129.1235
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 603.2595 - val_loss: 2112.2085
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 599.4074 - val_loss: 2095.4729
Epoch 310/10000
1/1 [==============================] - 0s 14ms/step - loss: 595.6053 - val_loss: 2078.9111
Epoch 311/10000
1/1 [==============================] - 0s 13ms/step - loss: 591.8516 - val_loss: 2062.5244
Epoch 312/10000
1/1 [==============================] 

...

1/1 [==============================] - 0s 14ms/step - loss: 106.9940 - val_loss: 3.8019
Epoch 1153/10000
1/1 [==============================] - 0s 14ms/step - loss: 106.5124 - val_loss: 3.5407
Epoch 1154/10000
1/1 [==============================] - 0s 14ms/step - loss: 106.0417 - val_loss: 3.2887
Epoch 1155/10000
1/1 [==============================] - 0s 14ms/step - loss: 105.5826 - val_loss: 3.0317
Epoch 1156/10000
1/1 [==============================] - 0s 14ms/step - loss: 105.1218 - val_loss: 2.7996
Epoch 1157/10000
1/1 [==============================] - 0s 14ms/step - loss: 104.6664 - val_loss: 2.5339
Epoch 1158/10000
1/1 [==============================] - 0s 14ms/step - loss: 104.3467 - val_loss: 2.5074
Epoch 1159/10000
1/1 [==============================] - 0s 14ms/step - loss: 103.7962 - val_loss: 2.2389
Epoch 1160/10000
1/1 [==============================] - 0s 14ms/step - loss: 103.7568 - val_loss: 1.6929
Epoch 1161/10000
1/1 [==============================] - 0s 15ms/step - l

1/1 [==============================] - 0s 15ms/step - loss: 81.4532 - val_loss: 8.9111
Epoch 1231/10000
1/1 [==============================] - 0s 14ms/step - loss: 93.4313 - val_loss: 7.0860
Epoch 1232/10000
1/1 [==============================] - 0s 14ms/step - loss: 81.0868 - val_loss: 7.8535
Epoch 1233/10000
1/1 [==============================] - 0s 14ms/step - loss: 80.6746 - val_loss: 9.7452
Epoch 1234/10000
1/1 [==============================] - 0s 14ms/step - loss: 79.8404 - val_loss: 11.3937
Epoch 1235/10000
1/1 [==============================] - 0s 13ms/step - loss: 80.2638 - val_loss: 11.8129
Epoch 1236/10000
1/1 [==============================] - 0s 14ms/step - loss: 69.5870 - val_loss: 14.3683
Epoch 1237/10000
1/1 [==============================] - 0s 14ms/step - loss: 69.5313 - val_loss: 23.9759
Epoch 1238/10000
1/1 [==============================] - 0s 14ms/step - loss: 66.7020 - val_loss: 27.2914
Epoch 1239/10000
1/1 [==============================] - 0s 14ms/step - loss:

1/1 [==============================] - 0s 15ms/step - loss: 211.4048 - val_loss: 164.1664
Epoch 1308/10000
1/1 [==============================] - 0s 16ms/step - loss: 210.3836 - val_loss: 159.3698
Epoch 1309/10000
1/1 [==============================] - 0s 15ms/step - loss: 207.4624 - val_loss: 158.2526
Epoch 1310/10000
1/1 [==============================] - 0s 14ms/step - loss: 200.2168 - val_loss: 153.7281
Epoch 1311/10000
1/1 [==============================] - 0s 14ms/step - loss: 202.1570 - val_loss: 145.7168
Epoch 1312/10000
1/1 [==============================] - 0s 14ms/step - loss: 197.2867 - val_loss: 133.3095
Epoch 1313/10000
1/1 [==============================] - 0s 14ms/step - loss: 174.2145 - val_loss: 105.6665
Epoch 1314/10000
1/1 [==============================] - 0s 14ms/step - loss: 169.7031 - val_loss: 88.5396
Epoch 1315/10000
1/1 [==============================] - 0s 13ms/step - loss: 164.9243 - val_loss: 82.6727
Epoch 1316/10000
1/1 [==============================] - 

1/1 [==============================] - 0s 14ms/step - loss: 228.5779 - val_loss: 235.0183
Epoch 1385/10000
1/1 [==============================] - 0s 14ms/step - loss: 214.9478 - val_loss: 200.6393
Epoch 1386/10000
1/1 [==============================] - 0s 14ms/step - loss: 203.1836 - val_loss: 171.9225
Epoch 1387/10000
1/1 [==============================] - 0s 13ms/step - loss: 200.8418 - val_loss: 153.1032
Epoch 1388/10000
1/1 [==============================] - 0s 14ms/step - loss: 209.5593 - val_loss: 159.0879
Epoch 1389/10000
1/1 [==============================] - 0s 14ms/step - loss: 208.9246 - val_loss: 161.3628
Epoch 1390/10000
1/1 [==============================] - 0s 13ms/step - loss: 199.4860 - val_loss: 152.9066
Epoch 1391/10000
1/1 [==============================] - 0s 14ms/step - loss: 198.2891 - val_loss: 132.0918
Epoch 1392/10000
1/1 [==============================] - 0s 14ms/step - loss: 178.8044 - val_loss: 165.6572
Epoch 1393/10000
1/1 [==============================] 

1/1 [==============================] - 0s 13ms/step - loss: 164.5197 - val_loss: 35.9773
Epoch 1462/10000
1/1 [==============================] - 0s 14ms/step - loss: 164.1362 - val_loss: 33.5314
Epoch 1463/10000
1/1 [==============================] - 0s 14ms/step - loss: 198.3965 - val_loss: 55.0953
Epoch 1464/10000
1/1 [==============================] - 0s 13ms/step - loss: 188.3234 - val_loss: 62.9649
Epoch 1465/10000
1/1 [==============================] - 0s 14ms/step - loss: 153.7569 - val_loss: 48.5312
Epoch 1466/10000
1/1 [==============================] - 0s 14ms/step - loss: 147.0194 - val_loss: 44.4577
Epoch 1467/10000
1/1 [==============================] - 0s 14ms/step - loss: 180.1785 - val_loss: 51.5825
Epoch 1468/10000
1/1 [==============================] - 0s 14ms/step - loss: 153.9276 - val_loss: 37.8750
Epoch 1469/10000
1/1 [==============================] - 0s 13ms/step - loss: 109.3386 - val_loss: 24.1024
Epoch 1470/10000
1/1 [==============================] - 0s 13ms

In [37]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[189.357](test_target) - [[181.11116]](prediction) = 8.245839721679687


In [40]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [41]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.693210,-0.938459,0.382853,1.236143,-0.644859,-1.229053,-1.23139,-1.225123,-1.249086,1.148205,...,1.277766,1.308949,1.412346,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.414123,-0.446997,0.987585,-0.023123,-0.767579,0.008662,0.01340,0.000757,0.050227,0.140893,...,-0.114016,-0.190804,-0.643250,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0
2,-0.720913,1.385456,-1.370438,-1.213019,1.412438,1.220391,1.21799,1.224366,1.198859,-1.289098,...,-1.163750,-1.118144,-0.769096,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0


train_target:


,Acre - Consumo de Cimento (t)
0,121.146
1,174.705
2,178.322


test_input:


...

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.704977,0.401023,-0.568106,-1.099064,-0.414155,0.731364,0.557172,0.863927,0.163877,1.108405,...,0.964765,-1.482779,-2.260447,1.033247,1.639674,0.502837,2.461657,1.121007,-1.165178,-2.121943


test_target:


,Acre - Consumo de Cimento (t)
12,189.357


1/1 [==============================] - 0s 39ms/step
Error: 8.245839721679687




[178.32095336914062,
 174.85464477539062,
 186.34957885742188,
 171.48094177246094,
 149.3680419921875,
 140.54029846191406,
 145.45811462402344,
 148.65322875976562,
 151.07821655273438,
 181.1111602783203]

In [42]:
display(targets)
display(predictions)

[173.494,
 193.672,
 171.7092857142857,
 151.48662857142855,
 134.2230857142857,
 150.491,
 148.696,
 153.80599999999998,
 178.033,
 189.357]

[178.32095336914062,
 174.85464477539062,
 186.34957885742188,
 171.48094177246094,
 149.3680419921875,
 140.54029846191406,
 145.45811462402344,
 148.65322875976562,
 151.07821655273438,
 181.1111602783203]

In [43]:
mae = mean_absolute_error(predictions, targets)
mae

12.696585253906253

In [44]:
porcentage = mae/np.mean(targets)
porcentage

0.07718439054076587